In [ ]:
import torch
from model.transformer import Transformer
from utils.search import Beam, Greedy
from utils.test import prompt
from utils.functional import parameter_count, model_size, load_module, load_model, parse_config

In [ ]:
config = parse_config("experiment/config.txt", verbose=True)
maxlen = config["maxlen"]
dm, dk, dv, nhead, layers, dff = config["dm"], config["dk"], config["dv"], config["nhead"], config["layers"], config["dff"]
bias, dropout, eps, scale  = config["bias"], config["dropout"], config["eps"], config["scale"]
beam_width, alpha, search_eps, fast = config["beam_width"], config["alpha"], config["search_eps"], config["fast"]
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

In [ ]:
tokenizer = load_module(path="experiment/tokenizer.pt", verbose=True)
sos, eos, pad = tokenizer.getitem("<sos>", module="source"), tokenizer.getitem("<eos>", module="source"), tokenizer.getitem("<pad>", module="source")
en_vocab, de_vocab = tokenizer.vocab_size()
print(f"Number of input tokens: {en_vocab}\nNumber of output tokens: {de_vocab}")

In [ ]:
model = Transformer(en_vocab, de_vocab, maxlen, pad_id=pad, dm=dm, dk=dk, dv=dv, nhead=nhead, layers=layers, 
                    dff=dff,bias=bias, dropout=dropout, eps=eps, scale=scale)
model.to(device);

In [ ]:
load_model(model, path="experiment/model.pt", verbose=True, device=device)
print(f"Number of Trainable Paramaters: {parameter_count(model):.1f}M\nSize of Model: {model_size(model):.1f}MB")

In [ ]:
beam = Beam(sos, eos, maxlen, width=beam_width, fast=fast, alpha=alpha, eps=search_eps)
greedy = Greedy(sos, eos, maxlen, alpha=alpha, eps=search_eps)

In [ ]:
sequence = input("Enter in the sequence of text: ")
output = prompt(sequence, model, tokenizer, beam, early_stop=False, device=device)
print(f"Translation: {output}")